In [1]:
# Heavily adapted from https://pytorch.org/tutorials/beginner/introyt/trainingyt.html and https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html

import sys
import os
import torch
import torch.nn as nn
import datetime as dt
import argparse
import subprocess
import ray.cloudpickle as pickle
import numpy as np
import tempfile
import pandas as pd

from utils.dataset import VideoDataset
from ray import tune, train
from ray.train import Checkpoint, get_checkpoint
from ray.tune.schedulers import ASHAScheduler
from models.cnn_lstm.cnn_lstm import CNN_LSTM
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
from sklearn.metrics import f1_score
from pathlib import Path
from functools import partial

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

main_folder_path = os.getcwd()

MODEL_NAME = "cnn_lstm"
NOW = dt.datetime.now()
FILENAME = f"{NOW.strftime('%Y-%m-%d-%H-%M-%S')}"
SAVE_DIR = f"{main_folder_path}/models/cnn_lstm/saved_models"
DATA_FOLDER = "data"
INF = 100000000.
NUM_WORKERS = 8
NUM_CLASSES = 2
GPUS_PER_TRIAL = torch.cuda.device_count() if torch.cuda.is_available() else 0

timestamp = dt.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
writer = SummaryWriter(f"runs/rcnn_{timestamp}")

def train_model(
    config: dict,
    epochs: int,
):
    """
    Trains the model and saves the weights into a `.pt` file.

    Args:
        epochs (int): Number of epochs.
        filename (str): Filename to save the model to.
        writer (SummaryWriter): Writer for logs.
        config (dict): Ray Tune dictionary.

    Returns:
        None
    """
    writer = SummaryWriter(f"runs/cnn_lstm_{timestamp}")
    train_dataset = VideoDataset(
        root=f"{main_folder_path}/data/train", 
        clip_len=config["steps"]
    )
    train_loader = DataLoader(
        dataset=train_dataset, 
        batch_size=int(config["batch_size"]),
        num_workers=NUM_WORKERS
    )

    val_dataset = VideoDataset(
        root=f"{main_folder_path}/data/validation", 
        clip_len=config["steps"]
    )
    val_loader = DataLoader(
        dataset=val_dataset, 
        batch_size=int(config["batch_size"]),
        num_workers=NUM_WORKERS
    )

    model = CNN_LSTM(
        input_channels=int(config["input_channels"]),
        num_cnn_layers=int(config["num_cnn_layers"]),
        num_kernels=int(config["num_kernels"]),
        kernel_size=int(config["kernel_size"]),
        stride=1,
        padding="same",
        dropout_prob=float(config["dropout_prob"]),
        bias=False,
        num_lstm_layers=int(config["num_lstm_layers"]),
        hidden_size=int(config["hidden_size"]),
        num_classes=NUM_CLASSES,
        bidirectional=bool(config["bidirectional"]),
        input_shape=(224, 224),
        steps=int(config["steps"])
    )

    print(device)
    model = model.to(device)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=config["lr"])

    if torch.cuda.is_available() and torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)

    running_loss = 0.
    last_loss = 0.
    best_vloss = INF

    checkpoint = get_checkpoint()
    if checkpoint:
        with checkpoint.as_directory() as checkpoint_dir:
            data_path = Path(checkpoint_dir) / "data.pkl"
            with open(data_path, "rb") as fp:
                checkpoint_state = pickle.load(fp)
            start_epoch = checkpoint_state["epoch"]
            model.load_state_dict(checkpoint_state["net_state_dict"])
            optimizer.load_state_dict(checkpoint_state["optimizer_state_dict"])
    else:
        start_epoch = 0

    print(f"Starting training at epoch: {start_epoch}")
    for epoch in tqdm(range(start_epoch, epochs)):
        print(f"Epoch: {epoch}")
        model.train()
        collected_labels, collected_predictions = [], []
        for i, data in tqdm(enumerate(train_loader)):
            vid_inputs, labels = data["video"].to(device), data["target"].to(device)

            optimizer.zero_grad()
            output = model(vid_inputs)
            loss = loss_fn(output, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            collected_labels.append(labels.cpu())
            collected_predictions.append(output.argmax(dim=1).cpu())

            if i % 10 == 9:
                last_loss = running_loss / 10
                epoch_f1 = f1_score(torch.cat(collected_labels), torch.cat(collected_predictions), average="weighted")
                print(f"Batch: {i + 1}, Loss: {last_loss}, F1 score: {epoch_f1}")
                tb_x = epoch * len(train_loader) + i + 1

                writer.add_scalar("Loss/train", last_loss, tb_x)
                writer.add_scalar("F1 score/train", epoch_f1, tb_x)

                running_loss = 0.

        model.eval()
        with torch.no_grad():
            collected_labels, collected_predictions = [], []
            for i, vdata in enumerate(val_loader):
                vid_inputs, labels = vdata["video"].to(device), vdata["target"].to(device)
                output = model(vid_inputs)
                loss = loss_fn(output, labels)
                running_loss += loss.item()

                collected_labels.append(labels.cpu())
                collected_predictions.append(output.argmax(dim=1).cpu())
            val_f1 = f1_score(torch.cat(collected_labels), torch.cat(collected_predictions), average="weighted")
            print(f"Validation Loss: {loss.item()}, Validation F1 score: {val_f1}")

        avg_vloss = running_loss / (i + 1)
        print(f"Train Loss: {last_loss}, Val Loss: {avg_vloss}")

        writer.add_scalars("Training vs Validation Loss",
                           {"Train": last_loss, "Validation": avg_vloss},
                           epoch + 1)
        writer.add_scalars("Training vs Validation F1 Score",
                           {"Train": epoch_f1, "Validation": val_f1},
                           epoch + 1)
        writer.flush()

        if avg_vloss < best_vloss:
            best_vloss = avg_vloss


        checkpoint_data = {
            "epoch": epoch,
            "net_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict()
        }

        with tempfile.TemporaryDirectory() as checkpoint_dir:
            data_path = Path(checkpoint_dir) / "data.pkl"
            with open(data_path, "wb")as fp:
                pickle.dump(checkpoint_data, fp)
            
            checkpoint = Checkpoint.from_directory(checkpoint_dir)
            train.report({
                "loss": avg_vloss,
                "f1": val_f1
            }, checkpoint=checkpoint)
    
    print("Finished training")

/home/wilsonwid/.local/share/virtualenvs/dsa4266-project-RvivrvFB/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-06 02:52:22,353	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-11-06 02:52:23,127	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-11-06 02:52:23.286101: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-06 02:52:23.294725: E external/local_xla/xla/stream_execut

In [2]:
config = {
    "input_channels": 6,
    "num_cnn_layers": 8,
    "num_kernels": 32,
    "kernel_size": 10,
    "dropout_prob": 0.25,
    "num_lstm_layers":5,
    "hidden_size": 32,
    "bidirectional": False,
    "steps": 100,
    "batch_size": 1,
    "lr": 1e-5
}

In [3]:
os.getcwd()

'/home/wilsonwid/github-repos/dsa4266-project'

In [ ]:
train_model(config, 3)

/home/wilsonwid/.local/share/virtualenvs/dsa4266-project-RvivrvFB/lib/python3.11/site-packages/ray/train/_internal/session.py:652: UserWarning: `get_checkpoint` is meant to only be called inside a function that is executed by a Tuner or Trainer. Returning `None`.
  warnings.warn(


cuda
Starting training at epoch: 0


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 0


/home/wilsonwid/.local/share/virtualenvs/dsa4266-project-RvivrvFB/lib/python3.11/site-packages/torch/nn/modules/conv.py:720: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:1036.)
  return F.conv3d(


...

Batch: 500, Loss: 0.6926870346069336, F1 score: 0.4460021436227224


Batch: 510, Loss: 0.6922154664993286, F1 score: 0.4466513878278584


Batch: 520, Loss: 0.6927872121334075, F1 score: 0.4461538461538462


Batch: 530, Loss: 0.6933934569358826, F1 score: 0.4494150943396226


Batch: 540, Loss: 0.6932486355304718, F1 score: 0.4535797875661837
